lets import the dataset and create a dataframe

In [89]:
import numpy as numpy
import pandas as pd
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

merge two dataset based on title of the movie so its complexity decreases

In [90]:
movies=movies.merge(credits,on='title')

lets take out the data which is not adding too much details to our recommendation 

used coloumns are : genres,id,keyword,title,overview,caste and crew

In [91]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]


check the missing data

In [92]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

drop the missing data 

In [93]:
movies.dropna(inplace=True)

check for duplicate data

In [94]:
movies.duplicated().sum()

0

In [95]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

checking the genre column we notice that those are strings so its difficult for us to bring in list form hence convert it write a function of it such a way that it carries only the name we dont neeed id

In [96]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [97]:
movies['genres']=movies['genres'].apply(convert)

do the same thing for keywords column

In [98]:
movies['keywords']=movies['keywords'].apply(convert)

if we observe the cast we need top three cast and its char name only so do it in that way

In [99]:
import ast
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
                L.append(i['name'])
                counter=+1
        else:
             break
    return L

In [100]:
movies['cast']=movies['cast'].apply(convert3)

now fetch ionloy the director name from crew column 

In [101]:
import ast
def convert4(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [102]:
movies['crew']=movies['crew'].apply(convert4)

now lets convert the overview which is string to list using lamda function

In [103]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [104]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


now we will concatinate all the list together to form tags as discussed before that avoid space btw differnet word of the same person eg:  sam mendis is one name but model reads it as two different enetities so remove that space

In [105]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(' ','') for i in x])

In [106]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['crew']+movies['cast']

In [107]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [108]:
new_df=movies[['movie_id','title','tags']]

now remove the list and join the spaces in tags

In [109]:
new_df['tags']=new_df['tags'].apply(lambda x: " ".join(x))

C:\Users\ADM\AppData\Local\Temp\ipykernel_19832\684433085.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: " ".join(x))


In [110]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


recommaned to convert tags into lower case

In [111]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\ADM\AppData\Local\Temp\ipykernel_19832\4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


you observe example like dancing dance two different entities lets make it a common enetities by applying stemming for this we will use a NLP programing library nltk install it then use porter stem function

In [112]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [113]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [114]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\ADM\AppData\Local\Temp\ipykernel_19832\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


now we have to do text vectorisation for this to proceed : lets combine all the tags together and find out the most common occuring 5000(adj but lesser words makes it less complex for more accuracy) words and dont forget to ignore stop words in english which doesnt add value 

In [115]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [116]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')


convert the gotten frquencies to arrays when you fit and transform it

In [117]:
vecotrs=cv.fit_transform(new_df['tags']).toarray()

to find out the no of frequent words we have a function in cv

In [118]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zoo', 'zooeydeschanel', 'zoëkravitz'],
      dtype=object)

to find the distance btw each vector we are not using euglidian distance btw each other rather we will use cosine distancde por: whne its high dimentinal data better go with cosine distance to find similarity

In [121]:
from sklearn.metrics.pairwise import cosine_similarity

In [122]:
similarity=cosine_similarity(vecotrs)

if you directly sort it out it will loose the indexes hence we cant predict how much st movie is similar to 2nd only and sort it in terms of similarity

In [135]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances=similarity[movie_index]
    movie_index = sorted(list(enumerate(distances)),reverse=True,key = lambda x: x[1])[1:6]
    for i in movie_index:
        print(new_df.iloc[i[0]].title)

In [139]:
recommend('Batman')

Batman
Batman & Robin
Batman Returns
The R.M.
Batman


In [142]:
pip install pickle


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [145]:
import pickle
pickle.dump(new_df.to_dict(),open('movie_list_Dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))